<a href="https://colab.research.google.com/github/joelwwiggins/car_ml/blob/main/Copy_of_notebook_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import tensorflow as tf
import pandas as pd

In [2]:
#import csv from path
#drop first row and column
df1 = pd.read_csv('CSVLog_20221218_084434.csv', skiprows=1)


In [3]:
#drop first column
df1_col=df1.iloc[:,1:]
df=df1_col

In [4]:
#drop rows where the vehicle speed in column 2 is <1
df_high_speed=df[df.iloc[:,2]>1]
#drop duplicate rows
df_duplicate=df_high_speed.drop_duplicates()
#how many duplicate rows were dropped
print(len(df_high_speed)-len(df_duplicate))

69007


In [5]:
#convert to numpy array
df_np=df_duplicate.to_numpy()

#train test split
from sklearn.model_selection import train_test_split
X_train, X_test = train_test_split(df_np, test_size=0.2, random_state=False)

#normalize the x_train
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)

#reshape the data to be 3d
X_train_3d = X_train_scaled.reshape(-1,X_train_scaled.shape[0], X_train_scaled.shape[1],1)






In [8]:





#use a 2dconv model to predict the next value
from keras.models import Sequential
from keras.layers import Dense, Conv1D, Flatten, MaxPooling1D 
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Conv2D
from keras.layers import MaxPooling2D

from keras.preprocessing import image
from keras.models import Model


model_shape = X_train_3d.shape



nn_model = tf.keras.models.Sequential()
nn_model.build(input_shape=model_shape)

#add a single layer 2d conv layer, with a pooling layer then flatten the data
nn_model.add(tf.keras.layers.Conv2D(filters=32, kernel_size=(3,3), activation='relu', input_shape=model_shape[1:]))
nn_model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))

nn_model.add(tf.keras.layers.Flatten())

#add a dense layer with 128 neurons
nn_model.add(tf.keras.layers.Dense(units=128, activation='relu'))

#add a dense layer with 64 neurons
nn_model.add(tf.keras.layers.Dense(units=1, activation='relu'))

#compile the model
nn_model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])

#fit the model to prevent incompatiblity errors
nn_model.fit(X_train_3d, X_train_3d, epochs=1, batch_size=1)







#normalize the x_test
X_test_scaled = scaler.fit_transform(X_test)

#reshape the data to be 3d
X_test = X_test_scaled.reshape(-1,X_test_scaled.shape[0], X_test_scaled.shape[1],1)
#evaluate the model

nn_model.evaluate(X_test_scaled, X_test_scaled)



nn_model.summary()

ResourceExhaustedError: OOM when allocating tensor with shape[13726336,128] and type float on /job:localhost/replica:0/task:0/device:CPU:0 by allocator cpu [Op:AddV2]

In [41]:
#save the model
nn_model.save('model.h5')

In [42]:
import plotly.express as px
#reshape X_train_scaled ot 2d
X_train_scaled = X_train_scaled.reshape((X_train_scaled.shape[0], X_train_scaled.shape[1]))


reconstructions_test = nn_model.predict(X_train_scaled)
test_loss = tf.keras.losses.mae(reconstructions_test, X_train_scaled)


df_loss_test = pd.DataFrame(test_loss)
fig = px.histogram(df_loss_test, x=df_loss_test.iloc[:,0], nbins=50)
fig.update_layout(yaxis_type="log")
fig.show()

In [43]:
#clean outliers from df and retrain
df_loss= nn_model.predict(df)

#inverse transform df_loss
df_loss_inv = scaler.inverse_transform(df_loss)

inv_scaled_loss=tf.keras.losses.mae(df_loss_inv, df)

#add a column to df with the loss
df['loss']=inv_scaled_loss

#trend the loss
fig = px.line(df, x=df.index, y='loss')
fig.show()






In [ ]:
#clean outliers from df and retrain
df_loss= nn_model.predict(df_duplicate)

#inverse transform df_loss
df_loss_inv = scaler.inverse_transform(df_loss)

inv_scaled_loss=tf.keras.losses.mae(df_loss_inv, df_duplicate)

#add a column to df with the loss

df_duplicate['loss']=inv_scaled_loss





#trend the loss
fig = px.line(df_duplicate, x=df_duplicate.index, y='loss')
fig.show()

